# TP12 et TP13 : _Puissance 4_
Le but de ces TP est de créer un _Puissance 4_ où l'utilisateur peut jouer contre l'ordinateur. L'ordinateur appliquera une stratégie de type min-max avec une heuristique pour se limiter à une profondeur raisonnable.

## TP12 : mise en place du jeu et calculs des scores

Pour mettre en place un jeu, il faut que l'utilisateur interagisse avec le programme, pour lui donner des informations. Pour celà, on peut utiliser la fonction `input` qui prend en argument une chaîne de caractères qui sera affichée et renvoie ce que l'utilisateur a tapé jusqu'au retour chariot, _sous forme de chaîne de caractères_ (et sans le retour chariot).

Exemple d'utilisation:

In [ ]:
p = input('Quel est votre prénom ? ')
print('Bonjour ' + p + '.')

### Exercice 1 : somme de deux entiers
Écrire un programme qui demande deux entiers à l'utilisateur et affiche leur somme.

#### Interlude : afficher une configuration
D'après la page wikipedia du jeu [_Puissance 4_](https://fr.wikipedia.org/wiki/Puissance_4), la grille de jeu est constituée de 6 rangées de 7 colonnes. Nous allons représenter chaque rangée par un tuple de 7 cases contenant chacune 0, 1 ou 2 (selon qu'elle est libre ou qu'elle contient un pion de $J_1$ ou de $J_2$), et la suite de rangées par un tuple de 6 rangées, de haut en bas. (Inutile de regarder la page wikipedia, sauf si vraiment vous ne savez pas jouer à Puissance 4.)

Ainsi le tuple
```python
( 
    (0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0),
    (0, 0, 1, 0, 0, 0, 0),
    (0, 0, 2, 1, 2, 0, 0),
    (0, 1, 2, 2, 1, 0, 0),
    (2, 1, 1, 1, 2, 2, 0)
)
```

représente la configuration

<a title="User François Haffner on fr.wikipedia, Public domain, via Wikimedia Commons" href="https://commons.wikimedia.org/wiki/File:Puissance4_01.svg"><img width="200" alt="Puissance4 01" src="https://upload.wikimedia.org/wikipedia/commons/thumb/d/dc/Puissance4_01.svg/256px-Puissance4_01.svg.png"></a>


si $J_1$ est représenté en orange et $J_2$ en rouge. **Dans toute la suite, on fera la supposition suivante: $J_1$ est le joueur humain et $J_2$ est l'ordinateur.**

Je vous fournis la fonction suivante qui permet d'afficher une configuration:

In [ ]:
from PIL import Image, ImageDraw

def affiche(g):
    """affiche la grille de jeu"""
    with Image.open("img/Puissance4_01.png").convert("RGBA") as base:
        h = base.height - (base.height%6)
        w = base.width - (base.width%7)
        couleur = [(255,255,255), (255, 165, 0), (255,0,0)]
        im = Image.new(size=base.size, mode='RGBA')
        for i in range(w):
            i2 = i // (w//7)
            for j in range(2, h):
                j2 = j // (h//6)
                im.putpixel((i, j), couleur[g[j2][i2]])
        im.paste(base, base)
        display(im)

In [ ]:
g = ( 
    (0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0),
    (0, 0, 1, 0, 0, 0, 0),
    (0, 0, 2, 1, 2, 0, 0),
    (0, 1, 2, 2, 1, 0, 0),
    (2, 1, 1, 1, 2, 2, 0)
    )
affiche(g)

### Exercice 2 : jouer un pion
Écrire une fonction `jouer` qui prend en argument une grille de _Puissance 4_ , un numéro de joueur `J` (supposé être 1 ou 2) et un numéro de colonne `c` (supposé entre 0 et 6) et renvoie un couple composé d'une **nouvelle** grille de jeu où le joueur `J` a joué dans la colonne `c` et de la ligne où se trouve le pion qu'il vient de jouer.
Si le joueur `J` ne peut pas jouer en colonne `c` (la colonne est pleine), la fonction doit renvoyer `None`.

Rappel: si `t` est un tuple, alors `list(t)` une liste avec les éléments de `t` dans le même ordre, et si `l` est une liste, `tuple(l)` est un tuple avec les éléments de `l` dans le même ordre.

In [ ]:
g = ( 
    (0, 0, 2, 0, 0, 0, 0),
    (0, 0, 1, 0, 2, 0, 0),
    (0, 0, 1, 0, 1, 0, 0),
    (0, 0, 2, 1, 2, 0, 0),
    (0, 1, 2, 2, 1, 0, 0),
    (2, 1, 1, 1, 2, 2, 0)
    )

g, i = jouer(g, 1, 5)
assert(g == ((0, 0, 2, 0, 0, 0, 0),
             (0, 0, 1, 0, 2, 0, 0),
             (0, 0, 1, 0, 1, 0, 0),
             (0, 0, 2, 1, 2, 0, 0),
             (0, 1, 2, 2, 1, 1, 0),
             (2, 1, 1, 1, 2, 2, 0)) and i == 4)

assert(jouer(g, 2, 2) == None)

g, i = jouer(g, 2, 6)
assert(g == ((0, 0, 2, 0, 0, 0, 0),
             (0, 0, 1, 0, 2, 0, 0),
             (0, 0, 1, 0, 1, 0, 0),
             (0, 0, 2, 1, 2, 0, 0),
             (0, 1, 2, 2, 1, 1, 0),
             (2, 1, 1, 1, 2, 2, 2)) and i == 5)

g, i = jouer(g, 1, 4)
assert(g == ((0, 0, 2, 0, 1, 0, 0),
             (0, 0, 1, 0, 2, 0, 0),
             (0, 0, 1, 0, 1, 0, 0),
             (0, 0, 2, 1, 2, 0, 0),
             (0, 1, 2, 2, 1, 1, 0),
             (2, 1, 1, 1, 2, 2, 2)) and i == 0)

### Exercice 3 : l'ordinateur joue au hasard
Nous allons faire une première version où l'ordinateur joue au hasard parmi les coup possibles.

1. Écrire une fonction `coups_possibles` qui renvoie la liste des colonnes d'une grille dans lesquelles on peu encore jouer.

In [ ]:
grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 0, 0),
          (2, 0, 2, 1, 2, 0, 0),
          (1, 1, 2, 2, 1, 0, 0),
          (2, 1, 1, 1, 2, 2, 0))
assert(coups_possibles(grille) == [1,2,3,4,5,6])

grille = tuple(tuple([1]*7) for _ in range(6))
assert(coups_possibles(grille) == [])

2. Écrire une fonction `coup_aleatoire` qui est une stratégie aléatoire pour un jour : elle prend en argument une grille et renvoie un numéro de colonne au hasard parmi les colonnes non pleines. Vous pouvez utiliser la fonction `choice` du module `random`.

3. Écrire une fonction `jeu_naif` qui prend en argument une stratégie (c'est-à-dire une fonction qui prend une grille en argument et renvoie un numéro de colonne à jouer pour $J_2$, en supposant que c'est effectivement à $J_2$ de jouer) et un premier joueur, et déroule le jeu jusqu'à ce que la grille soit pleine: pour le jour $J_1$, elle demande la colonne à jouer à l'utilisateur, pour le joueur $J_2$, elle utilise la stratégie fournie en argument.
Si $J_1$ entre `x`, la fonction doit s'arrêter.

Vous pouvez maintenant jouer contre une stratégie aléatoire (n'oubliez pas que votre moteur ne teste pas encore les alignements, donc quand vous en avez marre, pensez à entre `x` comme position):

In [ ]:
jeu_naif(coup_aleatoire, 1)

### Exercice 4 : tester si un coup est gagnant
Le jeu doit s'arrêter si un des deux joueurs gagne. Il faut donc écrire une fonction permettant de tester si l'ajout d'un pion mène à une position gagnante.

Toutes les fonctions qui suivent doivent prendre en argument une grille, et deux entiers correspondants respectivement au numéro de ligne et au numéro de colonne du dernier jeton placé dans la grille, et teste si ce jeton permet à la grille de vérifier un certain critère qui dépend de la fonction (on suppose, sans avoir à le vérifier qu'avant ce coup la grille ne vérifiait pas le critère).

1. Écrire une fonction `ligne` pour laquelle le critère est que le jeton qui vient d'être joué appartient à une suite horizontale d'au moins 4 jetons du même joueur.

In [ ]:
grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 0, 0),
          (2, 0, 2, 1, 2, 0, 0),
          (1, 1, 2, 2, 1, 0, 0),
          (2, 1, 1, 1, 2, 2, 0))
assert(ligne(grille, 0, 0) == False)

grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 0, 0),
          (2, 0, 2, 1, 0, 0, 0),
          (1, 1, 2, 2, 2, 2, 0),
          (2, 1, 1, 1, 2, 2, 0))
assert(ligne(grille, 4, 3) == True)

2. Écrire une fonction `colonne` pour laquelle le critère est que le jeton qui vient d'être joué appartient à une suite verticale d'au moins 4 jetons du même joueur.

In [ ]:
grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 0, 0),
          (2, 0, 2, 1, 2, 0, 0),
          (1, 1, 2, 2, 1, 0, 0),
          (2, 1, 1, 1, 2, 2, 0))
assert(colonne(grille, 0, 0) == False)

grille = ((0, 0, 0, 0, 0, 0, 0),
          (1, 0, 0, 0, 0, 0, 0),
          (1, 0, 2, 0, 0, 0, 0),
          (1, 0, 2, 1, 0, 0, 0),
          (1, 1, 2, 2, 2, 2, 0),
          (2, 1, 1, 1, 2, 2, 0))
assert(colonne(grille, 1, 0) == True)

3. Écrire une fonction `diagonale` pour laquelle le critère est que le jeton qui vient d'être joué appartient à une suite diagonale d'au moins 4 jetons du même joueur, où la diagonale est considérée avec les deux coordonnées qui varient de la même manière.

In [ ]:
grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 0, 0),
          (2, 0, 2, 1, 2, 0, 0),
          (1, 1, 2, 2, 1, 0, 0),
          (2, 1, 1, 1, 2, 2, 0))
assert(diagonale(grille, 2, 2) == False)

grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 1, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 0, 0),
          (2, 0, 2, 1, 2, 0, 0),
          (1, 1, 2, 2, 1, 0, 0),
          (2, 1, 1, 1, 2, 2, 0))
assert(diagonale(grille, 2, 2) == True)

grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 0, 0),
          (2, 0, 2, 1, 2, 0, 0),
          (1, 1, 2, 2, 1, 2, 0),
          (2, 1, 1, 1, 2, 2, 2))
assert(diagonale(grille, 3, 4) == False)

grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 2, 0, 0, 0),
          (2, 0, 2, 1, 2, 0, 0),
          (1, 1, 2, 2, 1, 2, 0),
          (2, 1, 1, 1, 2, 2, 2))
assert(diagonale(grille, 3, 4) == True)

4. Écrire une fonction `diagonale2` pour laquelle le critère est que le jeton qui vient d'être joué appartient à une suite diagonale d'au moins 4 jetons du même joueur, où la diagonale est considérée avec les deux coordonnées qui varient de manière opposée.

In [ ]:
grille = ((0, 0, 0, 0, 0, 0, 0),
          (0, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 2, 0),
          (2, 0, 2, 1, 2, 1, 0),
          (1, 1, 2, 2, 1, 2, 0),
          (2, 1, 1, 1, 2, 2, 0))
assert(diagonale2(grille, 4, 3) == False)

grille = ((0, 0, 0, 0, 0, 0, 0),
          (0, 0, 0, 0, 0, 0, 2),
          (1, 0, 1, 0, 0, 2, 2),
          (2, 0, 2, 1, 2, 1, 1),
          (1, 1, 2, 2, 1, 2, 2),
          (2, 1, 1, 1, 2, 2, 1))
assert(diagonale2(grille, 4, 3) == True)

5. Écrire une fonction `gagnant` pour laquelle le critère est que le jeton qui vient d'être joué appartient à une suite horizontale, verticale ou diagonale d'au moins 4 jetons du même joueur.

In [ ]:
g = ( 
    (0, 0, 0, 0, 0, 0, 0),
    (0, 0, 0, 0, 0, 0, 0),
    (0, 0, 1, 0, 0, 0, 0),
    (0, 0, 2, 1, 2, 0, 0),
    (0, 1, 2, 2, 1, 0, 0),
    (2, 1, 1, 1, 2, 2, 0)
    )
assert(gagnant(g, 2, 2) == False)

In [ ]:
g, _ = jouer(g, 1, 1)

c = 1
g, l = jouer(g, 2, c)

assert(gagnant(g, l, c) == True)

2. À partir de la fonction `jeu_naif`, écrire une fonction `jeu` qui s'arrête s'il y a un gagnant, en renvoyant le numéro du gagnant, 0 si la grille est pleine et -1 si l'utilisateur interrompt prématurément le déroulement avec un `x`. Votre fonction affichera aussi un message `"match nul"`, `"j'ai gagné!"` ou `"bravo à toi!"` selon la circonstance.

Vous pouvez maintenant essayer de battre la stratégie aléatoire:

In [ ]:
jeu(coup_aleatoire, 1)

----

## TP13 : algorithme min-max pour le _Puissance 4_
Il s'agit maintenant de construire une stratégie basée sur l'algorithme min-max. Bien sûr, on ne va pas construire tout l'arbre de jeu car il y a beaucoup trop de configurations. Nous allons donc mettre en place une heuristique permettant d'attribuer un score à une grille, et que nous utiliserons pour se limiter en profondeur dans l'arbre des configurations.

Rappel: **$J_1$ est le joueur humain et $J_2$ est l'ordinateur**.

Dans ce qui suit, on sépare le calcul du score d'une grille en regardant de façon indépendante les lignes, les colonnes, et les diagonales. On combinera ces scores pour les deux joueurs afin d'obtenir une heuristique pour $J_2$.

### Exercice 5 : score des colonnes
On considère une grille et un joueur $J$. On cherche à calculer un score associé aux colonnes pour ces données.

* Une colonne vide contribue pour 0 dans le score.
* Pour chaque colonne non vide:
   * si le jeton du dessus n'appartient pas à $J$, la colonne contribue pour 0 dans le score;
   * si le jeton du dessus appartient à $J$: soit $b$ le nombre d'emplacements sans jeton dans la colonne et $n$ la longueur du bloc supérieur de jetons appartenant à $J$ dans la colonne:
       * si $b+n < 4$, la colonne contribue pour 0 dans le score;
       * sinon si $n \geq 4$, la colonne contribue pour `math.inf` dans le score;
       * sinon la colonne contribue pour $n^2$ dans le score.
       
Écrire une fonction `score_colonne` qui prend une grille et un joueur en argument et renvoie ce score.

In [ ]:
grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 0, 0),
          (2, 0, 2, 1, 2, 0, 0),
          (1, 1, 2, 2, 1, 0, 0),
          (2, 1, 1, 1, 2, 2, 0))
assert(score_colonne(grille, 1) == 20)
assert(score_colonne(grille, 2) == 8)

### Exercice 6 : score des lignes
On considère une grille et un joueur $J$. On cherche à calculer un score associé aux lignes pour ces données.

* Une ligne vide contribue pour 0 dans le score.
* Pour chaque ligne non vide:
   * si la ligne contient au moins 4 jetons consécutifs appartenant à $J$, la ligne contribue pour `math.inf` dans le score;
   * pour chaque bloc de cases consécutives libres ou contenant un jeton appartenant à $J$: soit $b$ le nombre de cases libres de ce bloc et $n$ le nombre de cases appartenant à $J$ de ce bloc:
       * si $b+n < 4$ ou $n=0$, le bloc contribue pour 0 dans le score;
       * sinon le bloc contribue pour $(b+2n)^2$ au score.
       
Écrire une fonction `score_ligne` qui prend une grille et un joueur en argument et renvoie ce score.

In [ ]:
grille = ((1, 0, 0, 0, 0, 0, 0),
          (2, 0, 0, 0, 0, 0, 0),
          (1, 0, 1, 0, 0, 0, 0),
          (2, 0, 2, 1, 2, 0, 0),
          (1, 1, 2, 2, 1, 0, 0),
          (2, 1, 1, 1, 2, 2, 0))          
assert(score_ligne(grille, 1) == 217)
assert(score_ligne(grille, 2) == 150)

### Exercice 7 : score des diagonales
On considère une grille et un joueur $J$. On cherche à calculer un score associé aux diagonales pour ces données. Comme certaines diagonales varient avec $(+1,+1)$ et d'autres avec $(+1,-1)$, on va factoriser tout ça en créant une fonction intermédiaire qui calcule le score d'une seule diagonale.

Le principe du score d'une diagonale est le même que le principe du score d'une ligne, mais pour simplifier on ne considèrera que le bloc supérieur (attention: un bloc de pions peut avoir des cases vides des deux côtés sur une diagonale).

1. Écrire une fonction `score_une_diagonale` qui prend en argument une ligne, un joueur, un numéro de colonne, un numéro de ligne et un sens de variation de colonne ($+1$ ou $-1$) et renvoie le score associé à la diagonale en allant vers le bas.

2. Écrire des fonctions `score_diagonale` et `score_diagonale2` qui prennent en argument une grille et un joueur et renvoie la somme des scores associés à respectivement aux diagonales qui varient avec $(+1, +1)$ et à celles qui varient avec $(+1, -1)$.

### Exercice 8 : heuristique
L'heuristique associée à un couple (grille, joueur) est la différence entre le score associé au joueur (i.e. la somme des scores sur les lignes, les colonnes et les diagonales) et le double du score associé à son adversaire.

Écrire une fonction `heuristique` qui prend en argument une grille et un joueur et renvoie l'heuristique associée.

### Exercice 5 : algorithme min-max

Nous avons maintenant tout ce qu'il faut entre les mains pour écrire une fonction qui exécute l'algorithme min-max avec profondeur bornée pour le joueur $J_2$.

Le principe est de fabriquer l'arbre des configurations:
* si la configuration est terminale, on lui attribue le score `math.inf`, `-math.inf` ou `0` selon si $J_2$ gagne, $J_1$ gagne ou qu'il y a match nul;
* sinon si on est arrivé à la profondeur maximale qu'on s'est fixé, il faut évaluer le score de la configuration pour $J_2$ avec l'heuristique;
* sinon
    * si c'est à $J_1$ de jouer, on considère que $J-1$ va choisir le coup avec le score minimal pour $J_2$;
    * si c'est à $J_2$ de jouer, il va choisir le coup qui lui rapport le plus.

Pour chaque configuration, il faut donc retenir plusieurs renseignements qui seront conservés en mémoire dans un dictionnaire; à une configuration on pourra associer un triplet contenant dans cet ordre:
* le score de la configuration
* `True` si c'est un score exact, `False` s'il a été obtenu par l'heuristique, la configuration à atteindre s'il a été obtenu par `min` ou `max`
* la colonne à jouer pour atteindre la configuration suivante, ou `None` si ça n'a pas de sens.


Pour accélérer les calculs, on ne se relancera dans le calcul du score d'une configuration obtenue directement par heuristique que si on se retrouve effectivement dans cette configuration.

Écrire une fonction `min-max` aui prend en entrée une grille, un joueur, une profondeur, un tableau de scores sous forme d'un dictionnaire comme décrit ci-dessus et une heuristique, et renvoie le triplet décrit ci-dessus pour la grille et remplit le tableau de scores.

On définit une nouvelle stratégie:

In [ ]:
def strategie2(g):
    min_max(g, 2, 6, scores, heuristique1)
    return scores[g][2]

Et maintenant vous pouvez jouer pour de vrai:

In [ ]:
scores = {}
jeu(strategie2, 1)